<a href="https://colab.research.google.com/github/shrish83/Python-Basics/blob/master/BlackKofferAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Extraction an Text Analysis 

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
res = pd.read_excel('/content/cik_list.xlsx')
result = res.copy()
res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   CIK       152 non-null    int64         
 1   CONAME    152 non-null    object        
 2   FYRMO     152 non-null    int64         
 3   FDATE     152 non-null    datetime64[ns]
 4   FORM      152 non-null    object        
 5   SECFNAME  152 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 7.2+ KB


In [ ]:
res.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt


In [ ]:
res.shape

(152, 6)

In [ ]:
from bs4 import BeautifulSoup as bs
import urllib
from urllib.request import Request, urlopen
pre = 'https://www.sec.gov/Archives/'

## Extracting Derived variables

In [ ]:
!pip install --upgrade pip setuptools wheel
!pip install nltk

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
import statistics

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#import cookielib
contraining_words_whole_report = []

#Creating positive and negative dictionaries
master = pd.read_excel('/content/LoughranMcDonald_MasterDictionary_2018.xlsx')
pos = master['Word'].loc[master.Positive != 0]
neg = master['Word'].loc[master.Negative != 0]

#Number of StopWords
stopwords = open('/content/StopWords_GenericLong.txt','r+')
stopwords = list(bs(stopwords.read()))

#Uncertain and ConstrainDictionaries
uncertain = pd.read_excel('/content/uncertainty_dictionary.xlsx')
constrain = pd.read_excel('/content/constraining_dictionary.xlsx')
uncertain = list(uncertain['Word'])
constrain = list(constrain['Word'])

src = list(res['SECFNAME'])

def eval(df):
  #Get text
  url = pre+df['SECFNAME']
  hdr = {'User-Agent': 'Mozilla/5.0','Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'}
  req = Request(url,headers=hdr)
  try:
    global html
    html = urlopen(url).read()
  except urllib.error.HTTPError as e:
    print(e.reason)
  soup = bs(html)
  soup = soup.get_text()

    #Tokenize words and sentences
  word_token = word_tokenize(soup)
  sen_token = sent_tokenize(soup)

    #Cleaning using Stopwords
  cleanedData = [word for word in word_token if word not in stopwords]

    #Extracting derived variables
  pos_count = 0                       #postive score
  for word in pos:
    for w in cleanedData:
      if word == w:
        pos_count = pos_count +1

  neg_count = 0               #negative score
  for word in neg:
    for w in cleanedData:
      if word == w:
        neg_count = neg_count +1
    
    #polarity
  polarity = (pos_count-neg_count)/((pos_count+neg_count)+0.000001)

    #subjectivity
  subjective = (pos_count+neg_count)/(len(word_token)+0.000001)

    ###Analysis of Readability
  length_list = [len(word) for word in word_token]
  avg_sent_len = float(format(statistics.mean(length_list), '.2f'))
    
    #complex_word
  syllable_count = 0 
  complex_word = []
  for word in word_token:
    for w in word:
      if (w=='a' or w=='e' or w=='i' or w=='o' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U'):
        syllable_count=syllable_count+1
      if syllable_count > 2:
        complex_word.append(word)

  complex_word = list(set(complex_word))
  per_complex_word = len(complex_word)/len(word_token)

    #Fog Index
  fog_index = 0.4 * (avg_sent_len + per_complex_word)

    #Average number of words per sentence
  avg_num_words = len(word_token)/len(sen_token)

    #Constraining and Uncertainity Scores
  uncertain_score = len([word for word in word_token if word in uncertain])
  constrain_score = len([word for word in word_token if word in constrain])

    #positive word proportion
  pos_ratio = pos_count/len(word_token)

    #negative word proportion
  neg_ratio = neg_count/len(word_token)

    #constrain word proportion
  constrain_ratio = constrain_score/len(word_token)

    #uncertain word proportion
  uncertain_ratio = uncertain_score/len(word_token)
    
    #Constraining word for the whole report
  contraining_words_whole_report.append(constrain_score)

    #print(source,neg_ratio, constrain_score, polarity, fog_index)
  df['positive_score'] = pos_count
  df['negative_score'] = neg_count
  df['polarity_score'] = polarity
  df['average_sentence_length'] = avg_sent_len
  df['percentage_of_complex_words'] = per_complex_word
  df['fog_index'] = fog_index
  df['complex_word_count'] = len(complex_word)
  df['word_count'] = len(word_token)
  df['uncertainty_score'] = uncertain_score
  df['constraining_score'] = constrain_score
  df['positive_word_proportion'] = pos_ratio
  df['negative_word_proportion'] = neg_ratio
  df['uncertainty_word_proportion'] = uncertain_ratio
  df['constraining_word_proportion'] = constrain_ratio
  
  return df
  

In [ ]:
new = result.apply(eval,axis=1)

Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden
Forbidden


In [ ]:
#new['contraining_words_whole_report'] = contraining_words_whole_report
new.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,positive_score,negative_score,polarity_score,average_sentence_length,percentage_of_complex_words,fog_index,complex_word_count,word_count,uncertainty_score,constraining_score,positive_word_proportion,negative_word_proportion,uncertainty_word_proportion,constraining_word_proportion
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,edgar/data/3662/0000950170-98-000413.txt,40,176,-0.629630,4.46,0.039963,1.799985,8381,209718,13,78,0.000191,0.000839,0.000062,0.000372
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,edgar/data/3662/0000950170-98-001001.txt,26,70,-0.458333,4.47,0.047442,1.806977,6353,133910,16,85,0.000194,0.000523,0.000119,0.000635
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,26,70,-0.458333,4.47,0.047442,1.806977,6353,133910,16,85,0.000194,0.000523,0.000119,0.000635
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,edgar/data/3662/0000950170-98-002145.txt,17,99,-0.706897,4.46,0.068781,1.811512,7290,105989,13,43,0.000160,0.000934,0.000123,0.000406
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,0,1,-0.999999,4.37,0.328244,1.879298,473,1441,0,0,0.000000,0.000694,0.000000,0.000000


In [ ]:
new.shape

(152, 20)

In [ ]:
new.to_excel('output.xlsx')